In [ ]:
#pip install graphviz


Note: you may need to restart the kernel to use updated packages.


In [20]:
pip list | grep -i langchain

langchain                         1.1.0
langchain-classic                 1.0.0
langchain-community               0.4.1
langchain-core                    1.1.0
langchain-groq                    1.1.0
langchain-openai                  1.1.0
langchain-text-splitters          1.0.0
Note: you may need to restart the kernel to use updated packages.


In [1]:
import requests
from tavily import TavilyClient
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langgraph.graph import StateGraph, END

In [5]:
from dotenv import load_dotenv
import os

# Load .env (safe to call even if another cell calls it)
load_dotenv()

TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
if not TAVILY_API_KEY:
    raise EnvironmentError("TAVILY_API_KEY not found. Add it to your .env or environment variables.")

# Initialize Tavily client (TavilyClient was imported in a previous cell)
tavily = TavilyClient(api_key=TAVILY_API_KEY)

print("Tavily client initialized.")

Tavily client initialized.


## Create Tools

In [7]:


@tool
def tavily_search(query: str) -> str:
    """Search the web using Tavily."""
    try:
        response = tavily.search(query=query, max_results=5)
        return str(response)
    except Exception as e:
        return f"Tavily Error: {e}"

@tool
def arxiv_search(query: str) -> str:
    """Search academic papers from arXiv based on a topic."""
    try:
        url = f"http://export.arxiv.org/api/query?search_query=all:{query}&start=0&max_results=3"
        res = requests.get(url).text
        
        # Extract VERY simply (beginner-friendly) titles
        titles = []
        for line in res.split("\n"):
            if "<title>" in line and "arXiv" not in line:
                title = line.replace("<title>", "").replace("</title>", "").strip()
                titles.append(title)

        return str(titles[:3])
    except Exception as e:
        return f"ArXiv Error: {e}"

@tool
def publications_search(topic: str) -> str:
    """Search academic publications using CrossRef API."""
    try:
        url = f"https://api.crossref.org/works?query={topic}&rows=3"
        res = requests.get(url).json()
        items = res.get("message", {}).get("items", [])
        papers = []
        for item in items:
            title = item.get("title", ["No title"])[0]
            year = item.get("created", {}).get("date-parts", [[None]])[0][0]
            doi = item.get("DOI", "N/A")
            papers.append({"title": title, "year": year, "doi": doi})
        return str(papers)
    except Exception as e:
        return f"Publication Error: {e}"

@tool
def evaluate_info(text: str) -> str:
    """Evaluate the quality of the research."""
    return f"[Evaluation] Looks good. Helps support final report"


In [31]:
tools = [tavily_search, arxiv_search, publications_search, evaluate_info]

In [32]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

### Define Agent State

In [9]:
class ResearchState(dict):
    topic: str
    plan: list
    results: dict
    summary: str

##### NODES (Planner → Researcher → Evaluator → Reporter)

In [34]:
def planner(state: ResearchState):
    """LLM generates a multi-step research plan."""
    topic = state["topic"]
    prompt = f"""
            Generate a simple 3–5 step research plan for topic: {topic}
            Return steps as a numbered list.
            """
    plan_text = llm.invoke(prompt).content
    steps = [s.strip() for s in plan_text.split("\n") if s.strip()]

    state["plan"] = steps
    state["results"] = {}
    return state

def researcher(state: ResearchState):
    """Execute each plan step using Tavily, arXiv, CrossRef."""
    topic_results = {}

    for step in state["plan"]:
        web = tavily_search.invoke(step)
        arx = arxiv_search.invoke(step)
        pub = publications_search.invoke(step)

        topic_results[step] = {
            "web": web,
            "arxiv": arx,
            "crossref": pub,
        }

    state["results"] = topic_results
    return state

def evaluator(state: ResearchState):
    """Evaluate each step's results."""
    evals = {}

    for step, content in state["results"].items():
        text = str(content)
        evals[step] = evaluate_info.invoke(text)

    state["evaluation"] = evals
    return state



def reporter(state: ResearchState):
    """LLM writes the final research report."""
    prompt = f"""
Write a structured research report based on the following:

TOPIC:
{state['topic']}

PLAN:
{state['plan']}

RESEARCH RESULTS:
{state['results']}

EVALUATION:
{state['evaluation']}

Write it clearly with sections:
- Overview  
- Key Findings  
- Academic References  
- Insights  
- Conclusion  
"""

    final = llm.invoke(prompt).content
    state["summary"] = final
    return state

### Build Graph

In [35]:
graph = StateGraph(ResearchState)

graph.add_node("planner", planner)
graph.add_node("researcher", researcher)
graph.add_node("evaluator", evaluator)
graph.add_node("reporter", reporter)

graph.set_entry_point("planner")
graph.add_edge("planner", "researcher")
graph.add_edge("researcher", "evaluator")
graph.add_edge("evaluator", "reporter")
graph.add_edge("reporter", END)

research_agent = graph.compile()

In [36]:
topic = "Attention Mechanisms in Neural Networks"

output = research_agent.invoke({"topic": topic})

print("\n==========================================")
print("📘 FINAL MULTI-STEP RESEARCH REPORT")
print("==========================================")
print(output["summary"])

KeyError: 'evaluation'